<a href="https://colab.research.google.com/github/Kiwihead15/Test-models/blob/main/question_answer_Zephyr_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import chainlit as cl
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [19]:
config = {
    "max_new_tokens": 1024,
    "repetition_penalty": 1.1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

local_llm = CTransformers(model='C:\\Users\\pbiosca\\AI\\Projects\\models\\zephyr-7b-beta.Q6_K.gguf', config=config)   # Path to the local gguf file
#local_llm = CTransformers(model='TheBloke/zephyr-7B-beta-GGUF', model_file='zephyr-7b-beta.Q6_K.gguf', config=config)

In [20]:
template = """Question: {question}

Answer: Please refer to factual information and don't make up fictional data/information.
"""

In [21]:
prompt = PromptTemplate(template=template, input_variables=['question'])

In [22]:
llm_chain = LLMChain(prompt=prompt, llm=local_llm, verbose=True)

In [23]:
llm_chain("what is the capital city of Uruguay?")



> Entering new LLMChain chain...
Prompt after formatting:
Question: what is the capital city of Uruguay?

Answer: Please refer to factual information and don't make up fictional data/information.


> Finished chain.


{'question': 'what is the capital city of Uruguay?',
 'text': '\nThe correct answer is Montevideo, which is the largest city and capital of Uruguay, located in the southwestern region of the country. Other major cities in Uruguay include Salto, Paysandú, and Maldonado.'}

In [ ]:
@cl.on_chat_start
def main():
    prompt = PromptTemplate(template=template, input_variables=['question'])
    llm_chain = LLMChain(prompt=prompt, llm=local_llm, verbose=True)
    cl.user_session.set("llm_chain", llm_chain)

In [ ]:
@cl.on_message
async def main(message: str):
    llm_chain = cl.user_session.get("llm_chain")
    res = await llm_chain.acall(message, callbacks=[cl.AsyncLangchainCallbackHandler()])
    await cl.Message(content=res["text"]).send()

In [ ]:
! chainlit run <name-oython-script>.py -w
# The -w flag tells Chainlit to enable auto-reloading, so you don’t need to
# restart the server every time you make changes to your application. 
# Your chatbot UI should now be accessible at http://localhost:8000.